In [9]:
# !pip install gymnasium
# !pip install swig
# !pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379444 sha256=6f38c4624f86cbda817206e2db1a7d8a2a11113c227083b83fb5d0c66f312af6
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


# import 必要套件

In [20]:
import random
import time,math
import numpy as np
import gymnasium as gym
import gymnasium.wrappers as gym_wrap
import matplotlib.pyplot as plt
import matplotlib.animation as animation #輸出動畫影片
from IPython import display
from tqdm import tqdm

In [21]:
import torch
import torch.nn.functional as F
import collections
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [22]:
class ImageEnv(gym.Wrapper):
  def __init__(self,env,stack_frames=4,delay_op=50):
    super(ImageEnv, self).__init__(env)
    self.delay_op = delay_op
    self.stack_frames = stack_frames
  def reset(self):
    s, info = self.env.reset()
    for i in range(self.delay_op):
      s, r, terminated, truncated, info = self.env.step(0)
      s=s[:84, 6:90]/255.0
      self.stacked_state = np.tile( s , (self.stack_frames,1,1) )  # [4, 84, 84]
    return self.stacked_state, info

  def step(self, action):
    reward = 0
    for _ in range(self.stack_frames):
      s, r, terminated, truncated, info = self.env.step(action)
      if r==-100:terminated=True
      s=s[:84, 6:90]/255.0
      reward += r
      if terminated or truncated:break
      self.stacked_state = np.concatenate((self.stacked_state[1:], s[np.newaxis]), axis=0)
    return self.stacked_state, reward, terminated, truncated, info

# 建立Replay Buffer類別

In [23]:
class ReplayBuffer:
  def __init__(self,max_size=int(1e5), num_steps=1):
    self.s = np.zeros((max_size,4,84,84), dtype=np.float32)
    self.a = np.zeros((max_size,), dtype=np.int64)
    self.r = np.zeros((max_size, 1), dtype=np.float32)
    self.s_ = np.zeros((max_size,4,84,84), dtype=np.float32)
    self.done = np.zeros((max_size, 1), dtype=np.float32)
    self.ptr = 0
    self.size = 0
    self.max_size = max_size
    self.num_steps = num_steps

  def append(self,s,a,r,s_,done):
    self.s[self.ptr] = s
    self.a[self.ptr] = a
    self.r[self.ptr] = r
    self.s_[self.ptr] = s_
    self.done[self.ptr] = done
    self.ptr = (self.ptr + 1) % self.max_size
    self.size = min(self.size+1,self.max_size)
  def sample(self, batch_size):
    ind = np.random.randint(0, self.size, batch_size)
    return torch.FloatTensor(self.s[ind]),torch.LongTensor(self.a[ind]),torch.FloatTensor(self.r[ind]),torch.FloatTensor(self.s_[ind]),torch.FloatTensor(self.done[ind])

# 搭建DQN神經網路的類別

In [24]:
class DQN(torch.nn.Module):
  def __init__(self,n_act):
    super(DQN,self).__init__()
    self.conv1 = torch.nn.Conv2d(4, 16, kernel_size=8, stride=4)  #[N,4,84,84]->[N,16,20,20]
    self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=4, stride=2)  #[N,16,20,20]->[N,32,9,9]
    self.fc1 = torch.nn.Linear(32 * 9 * 9, 256)
    self.fc2 = torch.nn.Linear(256, n_act)
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = x.view((-1, 32 * 9 * 9))
    x = self.fc1(x)
    x = self.fc2(x)
    return x

# 設定是否載入模型參數，舊參數檔路徑，新參數檔路徑

In [25]:

Load_File=0
Old_File=f"Model-{Load_File}.pt"
if Load_File>0:
  Log= np.load(f"Log-{Load_File}.npy", allow_pickle=True).item()
else:
  Log={"TrainReward":[],"TestReward":[],"Loss":[]}

In [26]:
env=gym.make('CarRacing-v3',render_mode="rgb_array",domain_randomize=False, continuous=False)
env = gym_wrap.GrayscaleObservation(env)
env = ImageEnv(env)

# 搭建智能體Agent的類別

In [27]:
class DQNAgent():
  def __init__(self,gamma=0.9,eps_low=0.1,lr=0.00025):
    self.env = env
    self.n_act=self.env.action_space.n
    self.PredictDQN= DQN(self.n_act)
    self.TargetDQN= DQN(self.n_act)
    if Load_File>0:
      self.PredictDQN.load_state_dict(torch.load(Old_File))
      self.TargetDQN.load_state_dict(torch.load(Old_File))
    self.PredictDQN.to(device)
    self.TargetDQN.to(device)
    self.LossFun=torch.nn.SmoothL1Loss()
    self.optimizer=torch.optim.Adam(self.PredictDQN.parameters(),lr=lr)
    self.gamma=gamma
    self.eps_low=eps_low
    self.rb=ReplayBuffer(max_size=10000, num_steps=1)
  def PredictA(self,s):
    with torch.no_grad():
      return torch.argmax(self.PredictDQN(torch.FloatTensor(s).to(device))).item()
  def SelectA(self,a):
    return self.env.action_space.sample() if np.random.random()<self.EPS else a
  def Train(self,N_EPISODES):
    for i in tqdm(range(Load_File,N_EPISODES)):
      self.EPS=self.eps_low+(1-self.eps_low)*math.exp(-i*5/(N_EPISODES))
      total_reward=0
      s,_=self.env.reset()
      while True:
        a=self.SelectA(self.PredictA(s))
        s_,r,done,stop,_=self.env.step(a)
        self.rb.append(s,a,r,s_,done)
        if self.rb.size > 200 and i%self.rb.num_steps==0:self.Learn()
        if i % 20==0:  self.TargetDQN.load_state_dict(self.PredictDQN.state_dict())
        s=s_
        total_reward+=r
        if done or stop:break
      # print(f"\n{total_reward}")
      Log["TrainReward"].append(total_reward)
      if i % 10 == 9:
        test_reward=self.Test()
        print(f"\n訓練次數{i+1}，總回報{test_reward}")
        Log["TestReward"].append(test_reward)
        torch.save(self.PredictDQN.state_dict(), f"Model-{i+1}.pt")
        np.save(f"Log-{i+1}.npy", Log)
  def Learn(self):
    self.optimizer.zero_grad()
    batch_s, batch_a, batch_r, batch_s_, batch_done=self.rb.sample(32)
    predict_Q = (self.PredictDQN(batch_s.to(device))*F.one_hot(batch_a.to(device),self.n_act)).sum(1,keepdim=True)
    with torch.no_grad():
        # DoubleDQN 與 DQN 差在以下兩行，將動作選擇與價值評估分開
      a_ = self.PredictDQN(batch_s_.to(device)).max(dim=1)[1]
      target_Q = batch_r.to(device)+(1-batch_done.to(device))*self.gamma*(self.TargetDQN(batch_s_.to(device))*F.one_hot(a_,self.n_act)).sum(1)
      print(predict_Q.size(),target_Q.size())
    loss = self.LossFun(predict_Q, target_Q)
    Log["Loss"].append(float(loss))
    loss.backward()
    self.optimizer.step()
  def Test(self,VIDEO=False):
    total_reward=0
    video=[]
    s,_=self.env.reset()
    while True:
      video.append(self.env.render())
      a=self.PredictA(s)
      s,r,done,stop,_=self.env.step(a)
      total_reward+=r
      if done or stop:break
    if VIDEO:
      patch = plt.imshow(video[0]) #產生展示圖形物件
      plt.axis('off') #關閉坐標軸
      def animate(i): #設定更換影格的函數
        patch.set_data(video[i])
        #plt.gcf()=>建新繪圖區 animate=>更換影格函數 frames=>影格數 interval=>影隔間距(毫秒)
      anim = animation.FuncAnimation(plt.gcf(),animate,frames=len(video),interval=200)
      anim.save('Car_Racing.mp4') #儲存為mp4擋
    return total_reward
  def Record(self):
    total_reward=0
    s,_=self.env.reset()
    while True:
      image=self.env.render()
      plt.imshow(image)
      #plt.imsave(f"/content/drive/MyDrive/recording/{str(int(time.time()))}.png", image)
      a=self.PredictA(s)
      s,r,done,stop,_=self.env.step(a)
      print(r)
      total_reward+=r
      plt.pause(0.1)
      #清除目前的顯示
      display.clear_output(wait=True)
      if done or stop:break
    print(total_reward)

In [28]:
Agent=DQNAgent(gamma=0.95,eps_low=0.05,lr=0.00025)
Agent.Train(N_EPISODES=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:1040: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 

  0%|          | 1/1000 [00:17<4:49:41, 17.40s/it]

torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 

  0%|          | 1/1000 [00:19<5:27:17, 19.66s/it]

torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])
torch.Size([32, 1]) torch.Size([32, 32])


KeyboardInterrupt: 

In [ ]:
Agent.Record()

-64.69696969697041
